<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/pyAMPACT_underspecifiedScores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Underspecified Scores (Tony annotations)</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
import pandas as pd
import numpy as np
import librosa

Cloning into 'pyAMPACTtutorials'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 314 (delta 89), reused 18 (delta 14), pack-reused 177
Receiving objects: 100% (314/314), 17.62 MiB | 19.26 MiB/s, done.
Resolving deltas: 100% (150/150), done.
Importing libraries...


Load audio and symbolic files.

In [2]:
audio_file = '/content/Close to You vocals.wav'
csv_file = '/content/Close To You note track.csv'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(csv_file)

ConverterFileException: cannot find a format extensions for: /content/Close To You note track.csv

Run the alignment with pyampact's `run_alignment` function.

In [ ]:
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

# Visualize the alignment with True flag

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length, True)

Estimate performance paramters.

In [ ]:
# Calculate an instananeous frequency-gram of the audio signal
freqs, times, mags, f0_values, mags_mat = pyampact.ifgram(audiofile=audio_file, tsr=target_sr, win_ms=win_ms)
mags_db = librosa.amplitude_to_db(mags, ref=np.max)

# Estimate F0 values
f0_values, sig_pwr = pyampact.calculate_f0_est(audio_file, hop_length, win_ms, target_sr)
sig_pwr = mags ** 2 # power of signal, magnitude/amplitude squared

# Prune NaN and zero values from estimated F0 and power
f0_values = f0_values[~np.isnan(f0_values)]
sig_pwr = sig_pwr[sig_pwr != 0]

Compile estimated performance parameters

In [ ]:
pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, y, audio_file)

Export estimated performance parameters to a kern or MEI file

In [ ]:
piece.toKern()

'!!!COM: Mozart\n!!!OTL: Zwolf Variationen in C uber ein Menuett von Johann Christian Fischer\n**kern\t**kern\n*part2\t*part1\n*staff2\t*staff1\n*Ivox\t*Ivox\n*I"Part-2\t*I"Part-1\n*I\'P\t*I\'P\n=1\t=1\n*clefF4\t*clefG2\n*k[]\t*k[]\n*M3/4\t*M3/4\n4C\t2gg\n4E\t.\n4C\t16ffLL\n.\t16ee\n.\t16dd\n.\t16ccJJ\n=2\t=2\n4D\t8ggL\n.\t8ffJ\n4BB\t4.dd\n4GG\t.\n.\t8ff\n=3\t=3\n8DL\t12eeL\n.\t12gg\n8FJ\t.\n.\t12aaJ\n4G\t4cc\n4GG\t4b\n=4\t=4\n4C\t4b\n4GG\t4cc\n4CC\t4r\n==\t==\n*-\t*-\n!!!RDF**kern: %=rational rhythm\n!!!RDF**kern: l=long note in original notation\n!!!RDF**kern: i=editorial accidental\n!!!ONB: Translated from a krn file on 2024-03-14 via pyAMPACT\n!!!title: @{OTL}'

In [ ]:
piece.toMEI()